# Import Section

In [149]:
import pandas as pd
import numpy as np
import math

# Helper Class Definitions

In [150]:
class Criterion:
    def __init__(self, feature, threshold, target):
        self.feature = feature
        self.threshold = threshold
        self.target = target
    
    def get_feature(self):
        return self.feature
    
    def get_threshold(self):
        return self.threshold
    
    def get_target(self):
        return self.target

class Node:
    def __init__(self, criterion):
        self.left = None
        self.right = None
        self.criterion = criterion
        
    def set_left(self, node):
        self.left = node
        
    def set_right(self, node):
        self.right = node
        
    def set_criterion(self, criterion):
        self.criterion = criterion

    def get_left(self):
        return self.left

    def get_right(self):
        return self.right

    def get_criterion(self):
        return self.criterion

# Helper Function Definitions

## Find the best splitting asset
Iterate through all available features.  
Take median value as the threshold for each feature.  
Find the smallest impurity value and return it's feature and threshold value.  

In [151]:
def best_split_class(x, y, features, targets):
    imp_best = 1
    feature_best = features[0]
    threshold_best = 0
    # iterate through features
    for feature in features:
        # sort possible values of the selected feature
        vals = x[feature]
        vals = sorted(set(vals))
        # for i in range(1, len(vals)):
        # threshold = (vals[i - 1] + vals[i]) / 2.0
        # select median as the threshold
        threshold = pd.Series(vals).median()
        # iterate through targets
        for target in range(targets):
            d1pos = 0
            d1neg = 0
            d2pos = 0
            d2neg = 0
            # count positive and negative instances in 2 subsets
            for index, row in x.iterrows():
                #print([index, row[feature], threshold, y[index]])
                if row[feature] < threshold:
                    if y[index] == target:
                        d1pos = d1pos + 1
                    else:
                        d1neg = d1neg + 1
                else:
                    if y[index] == target:
                        d2pos = d2pos + 1
                    else:
                        d2neg = d2neg + 1
            # if target count is 0, skip
            if d1pos + d2pos == 0 or d1neg + d2neg == 0:
                continue
            # calculate entropy for 2 subsets
            d1 = d1pos + d1neg
            d2 = d2pos + d2neg
            if d1 == 0 or d2 == 0:
                continue
            if d1pos == 0 or d1neg == 0:
                imp1 = 0
            else:
                imp1 = -(d1pos / d1) * math.log(d1pos / d1, 2) - (d1neg / d1) * math.log(d1neg / d1, 2)
            if d2pos == 0 or d2neg == 0:
                imp2 = 0
            else:
                imp2 = -(d2pos / d2) * math.log(d2pos / d2, 2) - (d2neg / d2) * math.log(d2neg / d2, 2)
            # calculate new entopy for that entire set
            imp = (d1 / (d1 + d2)) * imp1 + (d2 / (d1 + d2)) * imp2
            # check for minimum entropy
            if imp < imp_best:
                # update criterion
                imp_best = imp
                feature_best = feature
                threshold_best = threshold
    return [feature_best, threshold_best]

## Grow tree recursively
If x is homogeneous, add a new leaf node to the decision tree.  
If maximum tree depth is reached, add a new leaf node with the most frequent value in y.  
Otherwise, find the best splitting feature, add it to the decision tree and return the tree.  

In [152]:
def grow_tree(x, y, features, targets, height):
    # check homogeneity
    if y.nunique() == 1:
        return Node(Criterion(None, None, y[0]))
    
    # check height
    if height == 0:
        return Node(Criterion(None, None, y.value_counts().idxmax()))
    
    # get best split criterion
    [feature, threshold] = best_split_class(x, y, features, targets)
    node = Node(Criterion(feature, threshold, -1))
    
    # split according to the criterion
    x1 = pd.DataFrame(columns = list(x.columns))
    x2 = pd.DataFrame(columns = list(x.columns))
    y1 = pd.Series([], dtype=np.float64)
    y2 = pd.Series([], dtype=np.float64)
    for index, row in x.iterrows():
        if row[feature] < threshold:
            x1 = x1.append(row, ignore_index=True)
            y1 = y1.append(pd.Series([y[index]], dtype=np.float64), ignore_index=True)
        else:
            x2 = x2.append(row, ignore_index=True)
            y2 = y2.append(pd.Series([y[index]], dtype=np.float64), ignore_index=True)
            
    # if left set is not empty
    if not x1.empty:
        node.set_left(grow_tree(x1, y1, features, targets, height - 1))
    if not x2.empty:
        node.set_right(grow_tree(x2, y2, features, targets, height - 1))
    return node

## Train from the given data and target.

In [153]:
def train(x, y, height):
    return grow_tree(x, y, list(x.columns), y.nunique(), height)

## Predict from the given data and decision tree

In [154]:
def predict(node, x):
    # check if it is a leaf node
    target = node.get_criterion().get_target()
    if target != -1:
        x['result'] = [target] * len(x.index)
        return x
    
    # split set according to the criterion
    x1 = pd.DataFrame(columns = list(x.columns))
    x2 = pd.DataFrame(columns = list(x.columns))
    feature = node.get_criterion().get_feature()
    threshold = node.get_criterion().get_threshold()
    for index, row in x.iterrows():
        if (row[feature] < threshold):
            x1 = x1.append(row, ignore_index=True)
        else:
            x2 = x2.append(row, ignore_index=True)
            
    # predict left subset
    if node.get_left() is not None and not x1.empty:
        x1 = predict(node.get_left(), x1)
    # predict right subset
    if node.get_right() is not None and not x2.empty:
        x2 = predict(node.get_right(), x2)
        
    # return 2 predicted subsets combined
    return x1.append(x2)

# Train and Predict Titanic Data Set
'Name' field is removed from the data set in preprocessing.  
'Sex' field is categorical in the data set, which is converted to integers (0/1) in preprocessing.  
The data set does not contain any N/A values.

In [160]:
# get data and preprocess it
df = pd.read_csv('titanic.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.drop(['Name'],axis=1)
mapping = {'male': 0, 'female': 1}
df['Sex'] = df['Sex'].map(mapping)
df.applymap(lambda s: mapping.get(s) if s in mapping else s)

# format training data for decision-tree
x = df.drop(['Survived'],axis=1)
y = df['Survived']

# split train and test data (80/20)
mask = np.random.rand(len(x)) < 0.8
train_x = x[mask]
train_y = y[mask]
test_x = x[~mask]
test_y = y[~mask]

### train & predict at various tree depth
for depth in range(0, 21, 5):
    ### train
    columns = list(x.columns)
    tree = train(train_x, train_y, depth)
    
    ### predict using train data
    data1 = train_x.copy()
    data1['target'] = train_y.tolist()
    data2 = predict(tree, train_x.copy())
    # sort for comparison
    data1 = data1.sort_values(by=columns)
    data2 = data2.sort_values(by=columns)
    # get target and prediction
    target = data1['target'].tolist()
    result = data2['result'].tolist()
    # compare and get the accuracy
    count_pos = 0
    count_neg = 0
    for i in range(len(target)): 
        if (target[i] == result[i]):
            count_pos = count_pos + 1
        else:
            count_neg = count_neg + 1
    accuracy_train = count_pos / (count_pos + count_neg) * 100

    ### predict using test data
    data1 = test_x.copy()
    data1['target'] = test_y.tolist()
    data2 = predict(tree, test_x.copy())
    # sort for comparison
    data1 = data1.sort_values(by=columns)
    data2 = data2.sort_values(by=columns)
    # get target and prediction
    target = data1['target'].tolist()
    result = data2['result'].tolist()
    # compare and get the accuracy
    count_pos = 0
    count_neg = 0
    for i in range(len(target)): 
        if (target[i] == result[i]):
            count_pos = count_pos + 1
        else:
            count_neg = count_neg + 1
    accuracy_test = count_pos / (count_pos + count_neg) * 100

    # print accuracy
    print([depth, accuracy_train, accuracy_test])

[0, 62.90550070521862, 55.61797752808989]
[5, 81.80535966149506, 76.40449438202246]
[10, 91.3963328631876, 77.52808988764045]
[15, 97.03808180535967, 75.84269662921348]
[20, 97.32016925246828, 75.28089887640449]


## Accuracy Report
When the tree depth is increased from 0 to 20, prediction accuracy is increased for training data, whereas it is decreased for test data at 15 and onwards.  
This indicates that overfitting becomes apparent at tree depth ~10.  
In grow_tree() function, the median is always selected for the threshold.  
Instead of using the median value, a more sophisticated approach may be implemented to get better results at smaller tree depths.

# Train and Predict Car Evaluation Data Set
Car evaluation data set contains categorical fields only, which should all be converted to integers in preprocessing.  
No N/A values are found in the data set.

In [161]:
# get data and preprocess
df = pd.read_csv('car.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

# convert categorical 'buying', 'maint' fields to integers
mapping = {'vhigh': 0, 'high': 1, 'med': 2, 'low': 3}
df['buying'] = df['buying'].map(mapping)
df['maint'] = df['maint'].map(mapping)
df.applymap(lambda s: mapping.get(s) if s in mapping else s)

# convert categorical 'doors' fields to integers
mapping = {'2': 0, '3': 1, '4': 2, '5more': 3}
df['doors'] = df['doors'].map(mapping)
df.applymap(lambda s: mapping.get(s) if s in mapping else s)

# convert categorical 'persons' fields to integers
mapping = {'2': 0, '4': 1, 'more': 2}
df['persons'] = df['persons'].map(mapping)
df.applymap(lambda s: mapping.get(s) if s in mapping else s)

# convert categorical 'lug_boot' fields to integers
mapping = {'small': 0, 'med': 1, 'big': 2}
df['lug_boot'] = df['lug_boot'].map(mapping)
df.applymap(lambda s: mapping.get(s) if s in mapping else s)

# convert categorical 'safety' fields to integers
mapping = {'low': 0, 'med': 1, 'high': 2}
df['safety'] = df['safety'].map(mapping)
df.applymap(lambda s: mapping.get(s) if s in mapping else s)

# convert categorical 'class' fields to integers
mapping = {'unacc': 0, 'acc': 1, 'good': 2, 'vgood': 3}
df['class'] = df['class'].map(mapping)
df.applymap(lambda s: mapping.get(s) if s in mapping else s)

# format training data for decision-tree
x = df.drop(['class'],axis=1)
y = df['class']

# split train and test data (80/20)
mask = np.random.rand(len(x)) < 0.8
train_x = x[mask]
train_y = y[mask]
test_x = x[~mask]
test_y = y[~mask]

### train & predict at various tree depth
for depth in range(0, 21, 5):
    ### train
    columns = list(x.columns)
    tree = train(train_x, train_y, depth)
    
    ### predict using train data
    data1 = train_x.copy()
    data1['target'] = train_y.tolist()
    data2 = predict(tree, train_x.copy())
    # sort for comparison
    data1 = data1.sort_values(by=columns)
    data2 = data2.sort_values(by=columns)
    # get target and prediction
    target = data1['target'].tolist()
    result = data2['result'].tolist()
    # compare and get the accuracy
    count_pos = 0
    count_neg = 0
    for i in range(len(target)): 
        if (target[i] == result[i]):
            count_pos = count_pos + 1
        else:
            count_neg = count_neg + 1
    accuracy_train = count_pos / (count_pos + count_neg) * 100

    ### predict using test data
    data1 = test_x.copy()
    data1['target'] = test_y.tolist()
    data2 = predict(tree, test_x.copy())
    # sort for comparison
    data1 = data1.sort_values(by=columns)
    data2 = data2.sort_values(by=columns)
    # get target and prediction
    target = data1['target'].tolist()
    result = data2['result'].tolist()
    # compare and get the accuracy
    count_pos = 0
    count_neg = 0
    for i in range(len(target)): 
        if (target[i] == result[i]):
            count_pos = count_pos + 1
        else:
            count_neg = count_neg + 1
    accuracy_test = count_pos / (count_pos + count_neg) * 100

    # print accuracy
    print([depth, accuracy_train, accuracy_test])

[0, 69.67509025270758, 71.42857142857143]
[5, 84.04332129963899, 82.79883381924198]
[10, 99.49458483754513, 97.667638483965]
[15, 100.0, 97.95918367346938]
[20, 100.0, 97.95918367346938]


## Accuracy Report
When the tree depth is increased from 0 to 20, prediction accuracy is increased for both the train and test data, and it saturated at 15 and onwards, which may indicate that the overfitting is not so apparent compared to the Titanic data set.  
Compared to Titanic data set which contains discrete values, Car data set contains categorical values only, which may have contributed to higher accuracies at similar tree depth.